# Output Parsers

Output Parsers are responsible for taking the output of an LLM and parsing into more structured format.

Features:
- parse text from message objects
- use output parsers to parse an LLM response into structured format
- parse JSON output
- parse XML output
- parse YAML output
- retry when output parsing errors occur
- try to fix errors in output parsing
- write a custom output parser class


## parse text from message objects

In [ ]:
from langchain_ollama import ChatOllama,  OllamaLLM

# WARNING: only specific models return a response with structured reasoning
# model = "gpt-oss:20b"         # no reasoning
# model = "deepseek-r1:8b"      # no reasoning
model = "magistral:24b"         # WORKS

llm = ChatOllama(base_url="http://localhost:11434", model=model, reasoning=True)

response = llm.invoke("Hello")
response

AIMessage(content='**Summary:**\nThe user presented the task as simply "Hello." Given the ambiguity, the most straightforward interpretation is that the task is to respond with or output "Hello."\n\nIn programming contexts, this could mean printing the string "Hello," which in Python would be:\n\n```python\nprint("Hello")\n```\n\nOutput:\n```\nHello\n```\n\nAlternatively, if interpreted as a greeting, the appropriate response would also be "Hello." Since no specific operation is requested on the word "Hello," the solution is to present or return "Hello."\n\nTherefore, the final answer is:\n\n\\boxed{Hello}', additional_kwargs={'reasoning_content': 'Alright, I have this problem statement:\n\n"Problem: Hello"\n\nFirst, I need to understand what\'s being asked. The term "Hello" is quite generic. It could be a greeting, or it might be part of a larger problem that wasn\'t fully provided. However, since the user just said "Hello," it seems like they are initiating contact rather than presen

In [ ]:
from langchain_core.tools import tool
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, ToolMessage, trim_messages

@tool
def get_weather(location: str) -> str:
    """Provides the weather from a location."""
    return "Sunny."

llm_with_tools = llm.bind_tools([get_weather], tool_choice="auto")

response = llm_with_tools.invoke("What's the weather in San Francisco, CA?")
response

AIMessage(content='### Summary\n\nI used the `get_weather` function with San Francisco, CA as the location parameter to fetch the current weather data. This approach ensures accuracy by specifying both the city and state.\n\nThe final answer is the weather information for San Francisco, CA, which would be presented after receiving the API response.\n\nHere\'s how I can call this function programmatically (though in reality, it would involve an actual API call):\n\n```python\nweather_data = get_weather(location="San Francisco, CA")\n```\n\nAnd then process and present that data to you. However, since we\'re in a simulated environment without direct internet access, the response would be structured as follows:\n\nThe current weather in San Francisco, CA is [weather description], with a temperature of [temperature]°C.\n\nBut since this is a simulation, I\'ll provide a placeholder answer based on typical weather patterns or assume you want to see how the call works. For actual use, you\'d 

In [46]:
from langchain_core.output_parsers import StrOutputParser

chain = llm_with_tools | StrOutputParser()

response = chain.invoke("What's the weather in San Francisco, CA?")
print(response)

To find out the current weather in San Francisco, CA, we can use an API or service that provides weather data. Assuming such a service is available, it would involve making a request to the service with the location specified (San Francisco, CA). The service would then return the current weather conditions for that location.

The final step would be to present this information in a clear and concise manner to the user.

Here's how we can do it:

1. Call the `get_weather` function with "San Francisco, CA" as the argument.
2. Wait for the response which should contain the current weather details such as temperature, precipitation chance, humidity, etc.
3. Format this information in a readable way for the user.

Let's proceed with that.




## write a custom output parser class

In [ ]:
from typing import Iterable
from langchain_ollama import ChatOllama
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

model = ChatOllama(base_url="http://localhost:11434", model="llama3.2:3b")

def parse(ai_message: AIMessage) -> str:
    """Parse the AI message."""
    return ai_message.content.swapcase()

chain = model | parse

messages = [
    SystemMessage(content="You are a helpful and very verbose assistant. You like to talk,talk , talk"),
    HumanMessage(content="What's could be the weather in San Francisco, CA?")
]

chain.invoke(messages)

'tHE cITY BY THE bAY! sAN fRANCISCO, cALIFORNIA IS KNOWN FOR ITS ICONIC FOGGY CLIMATE, BUT THAT DOESN\'T MEAN IT\'S ALWAYS GLOOMY, NO, NO, NO! tHE WEATHER IN sAN fRANCISCO CAN BE QUITE UNPREDICTABLE AND VARIED, MY FRIEND. lET ME TELL YOU ALL ABOUT IT!\n\nsAN fRANCISCO IS SITUATED ON THE pACIFIC COAST, WHICH MEANS IT\'S INFLUENCED BY THE COOL, MOIST AIR FROM THE pACIFIC oCEAN. tHIS LEADS TO A TYPICAL mEDITERRANEAN CLIMATE WITH MILD WINTERS AND COOL SUMMERS. bUT, AS i SAID, THE WEATHER CAN BE QUITE FINICKY.\n\niN THE SPRING (mARCH TO mAY), YOU MIGHT EXPECT MILD TEMPERATURES, RANGING FROM 50°f TO 65°f (10°c TO 18°c). tHE FOG CAN STILL BE PRESENT, BUT IT\'S NOT AS THICK AS DURING THE WINTER MONTHS. tHIS IS A WONDERFUL TIME TO VISIT sAN fRANCISCO IF YOU PREFER MILDER WEATHER AND FEWER TOURISTS.\n\nsUMMER DAYS IN sAN fRANCISCO USUALLY WARM UP TO THE MID-60S TO LOW 70S (18°c TO 22°c), WITH TEMPERATURES RARELY EXCEEDING 75°f (24°c). hOWEVER, IT\'S NOT UNCOMMON FOR THE TEMPERATURE TO DROP SIGNI

In [6]:
from langchain_core.runnables import RunnableGenerator
from langchain_core.messages import AIMessageChunk

def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    for chunk in chunks:
        yield chunk.content.swapcase()

streaming_parse = RunnableGenerator(streaming_parse)
chain = model | streaming_parse

chain.invoke(messages)

'tHE ENCHANTING CITY OF sAN fRANCISCO, cALIFORNIA! a PLACE OF BREATHTAKING NATURAL BEAUTY, RICH CULTURAL HERITAGE, AND... UNPREDICTABLE WEATHER! *WINK* aS WE EMBARK ON AN EXPLORATION OF THE METEOROLOGICAL POSSIBILITIES IN THIS FASCINATING CITY, i MUST SAY THAT THE bAY aREA\'S CLIMATE IS RENOWNED FOR ITS UNIQUE CHARACTERISTICS, WHICH CAN LEAD TO A MULTITUDE OF WEATHER SCENARIOS.\n\niN sAN fRANCISCO, cALIFORNIA, YOU CAN EXPECT TO ENCOUNTER A VARIETY OF ATMOSPHERIC CONDITIONS, DEPENDING ON THE TIME OF YEAR AND OTHER FACTORS. hERE ARE SOME POSSIBLE WEATHER SCENARIOS:\n\n1. **mILD AND SUNNY DAYS**: sAN fRANCISCO ENJOYS AN AVERAGE OF 283 SUNNY DAYS PER YEAR, WITH TEMPERATURES RANGING FROM THE MID-50S TO MID-70S fAHRENHEIT (13°c TO 23°c) DURING THE SUMMER MONTHS. iT\'S NOT UNCOMMON FOR THE CITY TO EXPERIENCE PLEASANT, WARM DAYS WITH GENTLE OCEAN BREEZES.\n2. **fOGGY AND COOL MORNINGS**: aH, YES! sAN fRANCISCO IS FAMOUS FOR ITS FOGGY CLIMATE, PARTICULARLY IN THE COASTAL AREAS. tHE CITY CAN BE 

In [7]:
for chunk in chain.stream("tell me about yourself in one sentence"):
    print(chunk, end="|", flush=True)

i| AM| A| HIGHLY| ADVANCED| LANGUAGE| MODEL|,| CAPABLE| OF| PROVIDING| INFORMATION| AND| ANSWERING| QUESTIONS| TO| THE| BEST| OF| MY| ABILITY| BASED| ON| MY| TRAINING| DATA|,| BUT| i| DON|'T| HAVE| PERSONAL| EXPERIENCES|,| EMOTIONS|,| OR| CONSCIOUSNESS| LIKE| HUMANS| DO|.||